In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import glob
import pickle
import helper_functions_06b as hf

In [2]:
DATE = ['20230302', '20230302', '20230302', '20230302', '20230302', '20230302', '20230302', '20230306', '20230309', '20230310']
SESSION = ['SM_Lek1', 'SM_Lek1', 'SM_Lek1', 'SM_Lek1', 'SE_Lek1', 'SE_Lek1', 'SE_Lek1', 'SE_Lek1', 'SM_Lek1', 'SM_Lek1']
DRONE = ['P1D1', 'P2D4', 'P3D5', 'P3D6', 'P1D1', 'P1D2', 'P3D5', 'P2D4', 'P3D5', 'P3D5']
VIDEOS = ['0874', '0573', '0152', '0545', '0885', '0672', '0163', '0707', '0396', '0432']
END_FRAMES = [505,4795,2325,3845,235,3060,1400,5100,2565,3170]

tracking_directory = '/Volumes/EAS_shared/blackbuck/working/processed/Field_Recording_2023/SpatialRegistration'

In [3]:
# Thresholds for translation, rotation, and scale
thresholds = {
    'T_translation': 30,  # e.g., 30 pixels
    'T_rotation': 20,     # e.g., 20 degrees
    'T_scale': 0.1         # e.g., 10% change
}

In [4]:
for date,session,drone,video,end_frame in zip(DATE,SESSION,DRONE,VIDEOS,END_FRAMES):
    internal_path = tracking_directory + '/' + date + '/' + session + '/' + drone
    file = internal_path + '/' + date + '_' + session + '_' + drone + '_DJI_' + video + '_YOLO_tracked.csv'
    
    print(date, session, drone, os.path.basename(file))
    
    df = pd.read_csv(file)
    df['x'] = df['bb_left'] + df['bb_width']/2
    df['y'] = df['bb_top'] + df['bb_height']/2
    df['idx'] = df['track_id']

    df = df.loc[:,['frame', 'x', 'y', 'idx']]

    # Step 1: Identify anchor frames
    anchor_frames = hf.identify_anchor_frames(df, thresholds)

    # Step 2: Find the best anchor for each frame
    df = hf.find_best_anchor_for_each_frame_parallel(df, anchor_frames)

    # Step 3: Apply homography using the best anchor for each frame
    df, homography_matrices, rms_errors = hf.apply_homography_to_best_anchor(df)

    anchor_filename = os.path.basename(file).replace('_YOLO_tracked.csv', '_Anchored.csv')
    pickle_filename = os.path.basename(file).replace('_YOLO_tracked.csv', '_homographies.pkl')
    df.to_csv(os.path.join(os.path.dirname(file), anchor_filename), index=False, mode='w')
    with open(os.path.join(os.path.dirname(file), pickle_filename), 'wb') as f:
        pickle.dump(homography_matrices, f)

20230306 SE_Lek1 P2D4 20230306_SE_Lek1_P2D4_DJI_0707_YOLO_tracked.csv
Not enough matching points for homography between frame 5055 and frame 5056.
Not enough matching points for homography between frame 5055 and frame 5057.
Not enough matching points for homography between frame 5055 and frame 5058.
Not enough matching points for homography between frame 5055 and frame 5059.
Not enough matching points for homography between frame 5055 and frame 5060.
Not enough matching points for homography between frame 5055 and frame 5061.
Not enough matching points for homography between frame 5055 and frame 5062.
Not enough matching points for homography between frame 5055 and frame 5063.
Not enough matching points for homography between frame 5055 and frame 5064.
Not enough matching points for homography between frame 5055 and frame 5065.
Not enough matching points for homography between frame 5055 and frame 5066.
Not enough matching points for homography between frame 5055 and frame 5067.
Not en